In [1]:
import pandas as pd
# Load in data 
train_split = pd.read_pickle("pandas_dumps/compounds_split.pkl")
all_data = pd.read_csv("activity_data.csv")

In [ ]:
all_data = all_data.merge(right=train_split,how="right")
# drop any record that has nan for f_avg_
all_data = all_data[all_data['f_avg_IC50'].notna()]
import numpy as np
all_data['pIC50'] = - all_data.f_avg_IC50.apply(np.log10)

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat